# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Question 3

### Construction of Toronto Neighbourhood analysis based on data from previous questions

The following notebook is the answer of the third question in the Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto.


First we retrieve all code from the 2 previous question to get back our dataframe:

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import geocoder
webcontent = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(webcontent,'lxml')
postaltable = soup.find('table')
postaltablecol = []
postaltablecontent = {}

for i, row in enumerate(postaltable.find_all('tr')):
    if i == 0:
        for Cval in row.find_all('th'):
            postaltablecol.append(Cval.text.replace('\n',''))
    else:
        postaltablecontent[i] =[row.find_all('td')[0].text ,  row.find_all('td')[1].text,  row.find_all('td')[2].text.replace('\n','')]

    
df = pd.DataFrame.from_dict(postaltablecontent,orient = 'index',columns = postaltablecol)
df.reset_index(inplace = True, drop = True)    
df = df[df['Borough']!= 'Not assigned']
df['Neighbourhood'].where(df['Neighbourhood'] != 'Not assigned',df['Borough'], inplace = True)
df = df.groupby(['Postcode','Borough']).Neighbourhood.unique().reset_index()
df['Neighbourhood']= df['Neighbourhood'].apply(', '.join)

df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.rename({'Postal Code': 'Postcode'},axis = 1, inplace = True)
df = pd.merge(df,df_geo,on='Postcode')

df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Next I import libraries from the lab I will need:

In [3]:
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


Next retrieve longitude and latitude of Toronto:

In [4]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


And the first visualization of Toronto and Neighbourhood:

In [5]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now to add data from foursquare, first getting the identifier:

In [6]:
#hopefully that is secure enough :)
with open ("foursquare_clientid.txt","r") as file1:
    CLIENT_ID = file1.read()
with open ("foursquare_secret.txt","r") as file2:
    CLIENT_SECRET = file2.read()
VERSION = '20180605'
LIMIT = 100

Now constructing the complete dataframe starting with code from the lab and modifying it  to only get food related venue because i'm interested in where I can eat.

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query=food&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        print(url)
       
         
        # make the GET request
        results = requests.get(url).json()["response"]["venues"]#['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        for v in results:
            try:
                venues_list.append([(
                name, 
                lat, 
                lng, 
                v['name'], 
                v['location']['lat'], 
                v['location']['lng'] , 
                v['categories'][0]['name'])])
            except:
                print('Problem with results category, will not be including this venue ')


        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

In [38]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rouge, Malvern
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.806686299999996,-79.19435340000001&query=food&radius=500&limit=100
Highland Creek, Rouge Hill, Port Union
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.7845351,-79.16049709999999&query=food&radius=500&limit=100
Guildwood, Morningside, West Hill
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.7635726,-79.1887115&query=food&radius=500&limit=100
Woburn
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=201806

Downsview West
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.7390146,-79.5069436&query=food&radius=500&limit=100
Downsview Central
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.7284964,-79.49569740000001&query=food&radius=500&limit=100
Downsview Northwest
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.7616313,-79.52099940000001&query=food&radius=500&limit=100
Victoria Village
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.725882299999995,-79.3155

Design Exchange, Toronto Dominion Centre
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.6471768,-79.38157640000001&query=food&radius=500&limit=100
Commerce Court, Victoria Hotel
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.6481985,-79.37981690000001&query=food&radius=500&limit=100
Bedford Park, Lawrence Manor East
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.7332825,-79.4197497&query=food&radius=500&limit=100
Problem with results category, will not be including this neighbourhood 
Roselawn
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU

Alderwood, Long Branch
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.60241370000001,-79.54348409999999&query=food&radius=500&limit=100
The Kingsway, Montgomery Road, Old Mill North
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.653653600000005,-79.5069436&query=food&radius=500&limit=100
Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
https://api.foursquare.com/v2/venues/search?&client_id=NQ0113PWV2JHPHANYV0B5IX4LBWTXHQ0TNU4YNZMQ5KIKBM1&client_secret=WE3IMFKNGWX13U1UHWC5LCZCMVXG55OUC2OKZVG1RJU0YY4X&v=20180605&ll=43.6362579,-79.49850909999999&query=food&radius=500&limit=100
Kingsway Park South West, Mimico NW, The Queensway West, Ro

In [39]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Rex Pak Food Packaging Ltd,43.805459,-79.194344,Business Service
1,"Rouge, Malvern",43.806686,-79.194353,Meena's Fine Foods,43.804476,-79.199753,Indian Restaurant
2,Cedarbrae,43.773136,-79.239476,CANBE Foods Inc,43.773546,-79.246082,Indian Restaurant
3,Cedarbrae,43.773136,-79.239476,Royal Kerala Foods,43.774473,-79.243008,Grocery Store
4,"Birch Cliff, Cliffside West",43.692657,-79.264848,Jatujak,43.688421,-79.270073,Thai Restaurant


In [40]:
toronto_venues.shape

(573, 7)

Now using one-hot encoding to get each Venue category as its own column.

In [51]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,African Restaurant,American Restaurant,Asian Restaurant,Bakery,Bar,Building,Burger Joint,Business Service,Butcher,...,Smoke Shop,Snack Place,South Indian Restaurant,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Tour Provider,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Birch Cliff, Cliffside West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


only interested in whether or not there is one of the food venues, not the percentage so I use the max methon instead of mean

In [81]:
toronto_grouped = toronto_onehot.groupby("Neighbourhood").max().reset_index()
toronto_grouped.head()

,Neighbourhood,African Restaurant,American Restaurant,Asian Restaurant,Bakery,Bar,Building,Burger Joint,Business Service,Butcher,...,Smoke Shop,Snack Place,South Indian Restaurant,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Tour Provider,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Adelaide, King, Richmond",1,0,1,0,0,1,0,0,0,...,0,0,0,0,1,1,1,0,1,0
1,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bathurst Manor, Downsview North, Wilson Heights",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now using K-Mean Cluster to try and create cluster of Neighbourhood

In [54]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 0, 0, 1, 0, 0, 0])

Doing some manipulation to get a dataframe with both neighbourhood position data and cluster info.

In [73]:
# add clustering labels
#toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_grouped.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.dropna(inplace = True)
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,African Restaurant,American Restaurant,Asian Restaurant,Bakery,...,Smoke Shop,Snack Place,South Indian Restaurant,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Tour Provider,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


And finally the map:

In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

This shows that the only place that stands out as far as type of food venues available is downtown Toronto with 4 clusters present there. The rest of the neighbourhood are all similar according to the K-mean clustering done previously. 

Thanks for reading!